<a href="https://colab.research.google.com/github/consumer-git/fake_news_detector/blob/main/fakenews_detector01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
from google.colab import drive
drive.mount('/content/d')

Drive already mounted at /content/d; to attempt to forcibly remount, call drive.mount("/content/d", force_remount=True).


In [105]:
import pandas as pd 
import numpy as np 

In [106]:
df_true=pd.read_csv('/content/d/MyDrive/Kaggle_competitons_try/fake_news_detector/archive/True.csv')
df_fake=pd.read_csv('/content/d/MyDrive/Kaggle_competitons_try/fake_news_detector/archive/Fake.csv')

In [107]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [108]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [109]:
df_fake['label']=np.zeros(len(df_fake.index),dtype=int)
df_true['label']=np.ones(len(df_true.index),dtype=int)

In [110]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [111]:
df_merge.shape

(44898, 5)

In [112]:
df_merge.label.value_counts()/len(df_merge.label) 

0    0.522985
1    0.477015
Name: label, dtype: float64

In [113]:
#shuffling dataset 
df_merge = df_merge.sample(frac = 1)


In [114]:
df_merge.head()

,title,text,subject,date,label
12823,HEY #NFL…CAN YOU HEAR US NOW? Monday Night Foo...,The #NFL has become a showcase for an anti-law...,politics,"Oct 5, 2016",0
22934,Boiler Room #102 – Tales From The Black Pill,Tune in to the Alternate Current Radio Network...,Middle-east,"March 24, 2017",0
8191,Jimmy Fallon Debuts His Bernie Sanders Impres...,The Tonight Show s Jimmy Fallon debuted his im...,News,"February 11, 2016",0
19050,"North Korea moving airplanes, boosting defense...",SEOUL (Reuters) - North Korea has been moving ...,worldnews,"September 26, 2017",1
23439,Syria Ceasefire Deal: A Cynical Ploy by Washin...,21st Century Wire says This latest diplomatic ...,Middle-east,"February 24, 2016",0


In [115]:
#cleaning the text 
import re 
import string 


stopwords_ms='/content/d/MyDrive/CAPSTONE_DATA/stop_words_long.txt'

def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(stopwords_ms, '', text)  
    return text  

In [116]:
df_merge['text']=df_merge['text'].apply(lambda x: wordopt(x))

In [117]:
df_merge['title']=df_merge['title'].apply(lambda x: wordopt(x))

In [118]:
df_merge.head()

,title,text,subject,date,label
12823,hey nfl can you hear us now monday night foo...,the nfl has become a showcase for an anti law...,politics,"Oct 5, 2016",0
22934,boiler room tales from the black pill,tune in to the alternate current radio network...,Middle-east,"March 24, 2017",0
8191,jimmy fallon debuts his bernie sanders impres...,the tonight show s jimmy fallon debuted his im...,News,"February 11, 2016",0
19050,north korea moving airplanes boosting defense...,seoul reuters north korea has been moving ...,worldnews,"September 26, 2017",1
23439,syria ceasefire deal a cynical ploy by washin...,century wire says this latest diplomatic deve...,Middle-east,"February 24, 2016",0


In [119]:
x = df_merge["text"]
y = df_merge["label"]

In [120]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [121]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25) 

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [123]:
xv_train

<33673x94819 sparse matrix of type '<class 'numpy.float64'>'
	with 6882922 stored elements in Compressed Sparse Row format>

In [124]:
vectorization.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)>

In [125]:
vectorization.vocabulary_

{'former': 30319,
 'republican': 70050,
 'presidential': 65370,
 'candidate': 12218,
 'mitt': 53958,
 'romney': 71708,
 'has': 35653,
 'been': 7426,
 'openly': 59790,
 'critical': 18538,
 'of': 59216,
 'donald': 23343,
 'trump': 86172,
 'and': 3167,
 'the': 83628,
 'party': 61712,
 'continued': 17297,
 'support': 81394,
 'bombastic': 9536,
 'billionaire': 8437,
 'for': 30133,
 'president': 65361,
 'now': 58390,
 'one': 59654,
 'his': 37055,
 'campaign': 12126,
 'operatives': 59820,
 'written': 93116,
 'stunning': 80672,
 'op': 59763,
 'ed': 24840,
 'in': 39413,
 'which': 91857,
 'he': 35946,
 'roundly': 71976,
 'trashed': 85646,
 'gave': 31864,
 'rather': 68020,
 'glowing': 32972,
 'endorsement': 25942,
 'democratic': 20896,
 'nominee': 57949,
 'hillary': 36856,
 'clinton': 15352,
 'that': 83600,
 'man': 50319,
 'is': 41274,
 'david': 19847,
 'nierenberg': 57543,
 'finance': 29242,
 'chair': 13623,
 'piece': 63341,
 'appeared': 3903,
 'on': 59635,
 'cnbc': 15555,
 'website': 91383,
 's

In [126]:
import seaborn as sns  

In [127]:
vectorization.vocabulary_.keys()

dict_keys(['former', 'republican', 'presidential', 'candidate', 'mitt', 'romney', 'has', 'been', 'openly', 'critical', 'of', 'donald', 'trump', 'and', 'the', 'party', 'continued', 'support', 'bombastic', 'billionaire', 'for', 'president', 'now', 'one', 'his', 'campaign', 'operatives', 'written', 'stunning', 'op', 'ed', 'in', 'which', 'he', 'roundly', 'trashed', 'gave', 'rather', 'glowing', 'endorsement', 'democratic', 'nominee', 'hillary', 'clinton', 'that', 'man', 'is', 'david', 'nierenberg', 'finance', 'chair', 'piece', 'appeared', 'on', 'cnbc', 'website', 'says', 'knows', 'her', 'stuff', 'she', 'emotionally', 'mature', 'centered', 'respects', 'enjoys', 'working', 'with', 'people', 'from', 'all', 'backgrounds', 'diplomatic', 'skills', 'needed', 'to', 'break', 'gridlock', 'washington', 'lead', 'our', 'country', 'well', 'america', 'needs', 'steady', 'hand', 'tiller', 'despite', 'being', 'ties', 'dating', 'back', 'had', 'this', 'say', 'about', 'current', 'standard', 'bearer', 'most', 'd

In [128]:
# from nltk import FreqDist
# def plot_word_frequency(words, top_n=10):
#     word_freq = FreqDist(words)
#     labels = [element[0] for element in word_freq.most_common(top_n)]
#     counts = [element[1] for element in word_freq.most_common(top_n)]
#     plot = sns.barplot(labels, counts)
#     plot.set_xticklabels(plot.get_xticklabels(),rotation=45)
#     return plot

In [129]:
# plot_word_frequency(df_merge.text, 15)

In [130]:
#assesing classifiers algorithms - scrapped this because it was taking too long 
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [145]:
pred_dt = DT.predict(xv_test)

In [146]:
DT.score(xv_test, y_test)

0.9955456570155902

In [148]:
type(xv_test)

scipy.sparse.csr.csr_matrix

In [149]:
type(y_test)

pandas.core.series.Series

In [150]:
DT.score(xv_train, y_train)

1.0

In [152]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5734
           1       1.00      0.99      1.00      5491

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

